In [1]:
import os
import time
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os

In [2]:
BATCH_SIZE = 100 # Edit this

In [11]:
test_link = 'https://krisha.kz/a/show/1000906432'
response = requests.get(test_link)
soup = BeautifulSoup(response.text, 'html.parser')

t_ = soup.find('script', id='jsdata')
a = t_.get_text(strip=True)
a.find('lat')

1720

In [12]:
print(a)
# "lat":43.206244,"lon":76.903154,"zoom":14
print(a[a.find('lat') + 5:a.find(',"lon"')])
print(a[a.find('lon') + 5:a.find(',"zoom"')])

window.data = {"advert":{"id":1000906432,"storage":"live","commentsType":"adv","isCommentable":false,"isCommentableByEveryone":false,"isOnMap":true,"hasPrice":true,"price":175000000,"photos":[{"src":"https://krisha-photos.kcdn.online/webp/b2/b27a4430-5c83-431a-a6ee-af3896785e65/1-full.jpg","w":576,"h":1024,"title":"Продажа участков в Алматы: Участок · 8 соток, Жарокова — Альфараби Жарокова за 175 млн 〒 — фото 2","alt":"Участок · 8 соток, Жарокова — Альфараби Жарокова за 175 млн 〒 в Алматы, Бостандыкский р-н — фото 2"},{"src":"https://krisha-photos.kcdn.online/webp/b2/b27a4430-5c83-431a-a6ee-af3896785e65/2-full.jpg","w":576,"h":1024,"title":"Продажа участков в Алматы: Участок · 8 соток, Жарокова — Альфараби Жарокова за 175 млн 〒 — фото 3","alt":"Участок · 8 соток, Жарокова — Альфараби Жарокова за 175 млн 〒 в Алматы, Бостандыкский р-н — фото 3"},{"src":"https://krisha-photos.kcdn.online/webp/b2/b27a4430-5c83-431a-a6ee-af3896785e65/3-full.jpg","w":576,"h":1024,"title":"Продажа участков в 

In [13]:
def parse_offer_page(url, proxies=None, headers=None):
    
    # Get the the page sourse
    response = requests.get(url, proxies=proxies, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to load page: {url}")

    # Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    result = {"Link": url}
    
    coord = soup.find('script', id='jsdata')
    if coord:
        a = coord.get_text(strip=True)
        
        result['lat'] = a[a.find('lat') + 5:a.find(',"lon"')]
        result['lon'] = a[a.find('lon') + 5:a.find(',"zoom"')]
    
    # Extract price
    price_div = soup.find('div', class_="offer__price")
    if price_div:
        t_price = price_div.get_text(strip=True)
        # t_price = t_price.replace("&nbsp;", "").strip()
        result['Price'] = t_price

    # Extract info items
    short_description = soup.find('div', class_='offer__short-description')
    if short_description:
        for item in short_description.find_all('div', class_='offer__info-item'):
            title_div = item.find('div', class_='offer__info-title')
            value_div = item.find('div', class_='offer__advert-short-info')

            if title_div and value_div:
                key = title_div.get_text(strip=True)
                value = value_div.get_text(strip=True)
                result[key] = value

    # Extract description
    description_div = soup.find('div', class_='js-description a-text a-text-white-spaces')
    if description_div:
        result['Описание'] = description_div.get_text(strip=True)

    return result

In [14]:
parse_offer_page("https://krisha.kz/a/show/1001366917?srchid=0196c698-0172-717b-85cd-96c79051ec14&srchtype=filter&srchpos=40077")

{'Link': 'https://krisha.kz/a/show/1001366917?srchid=0196c698-0172-717b-85cd-96c79051ec14&srchtype=filter&srchpos=40077',
 'lat': '53.142654',
 'lon': '63.62026',
 'Price': '7\xa0300\xa0000〒',
 'Город': 'Алтынсарино (Костанайский район), Костанайская обл.показать на карте',
 'Площадь участка': '14 соток',
 'Делимость': 'делимый',
 'Местоположение': 'в пригороде',
 'Целевое назначение': 'ИЖС (индивидуальное жилищное строительство)',
 'Коммуникации': 'свет, вода, газ, канализация, отопление, вентиляция',
 'Возможен обмен': 'Нет',
 'Описание': 'Фундамент, удобный въезд, выкуплен, все документы.\n\nПродается земельный участок в Мичуринском сельском округе, с. Алтынсарино, Альжанка, в 5 км от города со стороны КЖБИ. Земельный участок - 0, 1417 Га, это 14, 5 соток. Участок расположен на центральной улице. Целевое назначение для строительства индивидуальных жилых домов. На участке вырыт котлован, положен фундамент. Так же лежат плиты для строительства дома. Стройматериалы в продаже по договор

In [21]:
import json

import logging
logging.basicConfig(level=logging.INFO, filename="log/py_log.log", filemode="a", format="%(asctime)s %(levelname)s %(message)s")



data = []
link_srourse_file = '../parsing_results/output_links_270525.csv'
links_df = pd.read_csv(link_srourse_file)
links = list(links_df['Link'])
print(f'parsing links from path: {link_srourse_file}')


erro_pages = []


start = len(os.listdir('../parsing_results/json')) * BATCH_SIZE 
print(f'starting from {start} th line')
end = len(links)

for i in tqdm(range(start, end)):
    url = links[i]
    t_data = {}
    try:
        result = parse_offer_page(url)
        t_data['link'] = result["Link"]
        t_data['price'] = result['Price'] if 'Price' in result.keys() else None
        t_data['city'] = result['Город'] if 'Город' in result.keys() else None
        t_data['area'] = result['Площадь участка'] if 'Площадь участка' in result.keys() else None
        t_data['devidable'] = result['Делимость'] if 'Делимость' in result.keys() else None
        t_data['location'] = result['Местоположение'] if 'Местоположение' in result.keys() else None
        t_data['intended_purpose'] = result['Целевое назначение'] if 'Целевое назначение' in result.keys() else None
        t_data['communications'] = result['Коммуникации'] if 'Коммуникации' in result.keys() else None
        t_data['description'] = result['Описание'] if 'Описание' in result.keys() else None
        t_data['lat'] = result['lat'] if 'lat' in result.keys() else None
        t_data['lon'] = result['lon'] if 'lon' in result.keys() else None
        data.append(t_data)
        time.sleep(1)
        
        if i % BATCH_SIZE == 0 and i != 0:
            with open(f"../parsing_results/json/for_hb_{i // BATCH_SIZE}.json", "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            data = []
    except Exception as e:
        logging.error(e)
        erro_pages.append(url)

with open("../parsing_results/json/for_hb_last.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
    
with open("non_valid_links.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    for link in erro_pages:
        writer.writerow([link])
    
    

parsing links from path: ../parsing_results/output_links_270525.csv
starting from 23000 th line


100%|██████████| 8727/8727 [3:17:58<00:00,  1.36s/it]   


In [22]:
len(os.listdir('../parsing_results/json'))

314

In [ ]:
root = '../parsing_results/json'
df = pd.DataFrame()
for file in tqdm(os.listdir(root)):
    # print(file)
    df = pd.concat([df, pd.read_json(f'{root}/{file}', orient='records')], ignore_index=True)
    
print(df.shape)
df.to_json('../paring_results/for_hb_final_records.json', orient='records', index=False, indent=4, force_ascii=False)
df.info()

100%|██████████| 314/314 [00:06<00:00, 47.47it/s]


(30819, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30819 entries, 0 to 30818
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   link              30819 non-null  object
 1   price             30819 non-null  object
 2   city              30819 non-null  object
 3   area              30817 non-null  object
 4   devidable         19521 non-null  object
 5   location          24421 non-null  object
 6   intended_purpose  25170 non-null  object
 7   communications    20053 non-null  object
 8   description       29886 non-null  object
 9   lat               30819 non-null  object
 10  lon               30819 non-null  object
dtypes: object(11)
memory usage: 2.6+ MB


In [ ]:
# df = pd.read_json('../parsing_results/for_hb_final_records.json', orient='records')
# df.head()

,link,price,city,area,devidable,location,intended_purpose,communications,description
0,https://krisha.kz/a/show/1000787198?srchid=019...,107 000 000〒,"Казахстан, Алматинская обл.показать на карте",635 соток,делимый,в пригороде,КХ (крестьянское/фермерское хозяйство),"свет, вода, газ","Ровный, удобный въезд, выкуплен, все документы..."
1,https://krisha.kz/a/show/1001554653?srchid=019...,20 000 000〒,"Уральск, Западно-Казахстанская обл.показать на...",8 га,None,None,None,None,Продаётся земельный участок — 8 соток \nРайон ...
2,https://krisha.kz/a/show/1000265732?srchid=019...,7 600 000〒,"Шымкент, Абайский р-нпоказать на карте",8 соток,делимый,"в городе, вдоль трассы",ИЖС (индивидуальное жилищное строительство),"свет, вода, газ, канализация, вентиляция","Ровный, удобный въезд, выкуплен, госакт, все д..."
3,https://krisha.kz/a/show/1001240202?srchid=019...,2 000 000〒,"Актобе, жилой массив Рауанпоказать на карте",11.24 соток,неделимый,в пригороде,ИЖС (индивидуальное жилищное строительство),"свет, вода, газ","Ровный, удобный въезд, госакт, все документы, ..."
4,https://krisha.kz/a/show/1001553986?srchid=019...,2 200 000〒,"Жамбыл (Жамбылский район), Алматинская обл.пок...",20000 га,делимый,"в пригороде, вдоль трассы",ИЖС (индивидуальное жилищное строительство),"свет, вода",Ровный.\n\n1000.0 шаршы метрден 2 жер телімі қ...


In [4]:
df.shape

(30819, 11)

In [5]:
df.to_excel('../parsing_results/for_hb_final_30800.xlsx', index=False)